In [ ]:
# User Random Forest Regressor (extra-trees), model based on price and volume
# Extra-trees regressor implements a meta estimator that fits a number of randomized 
# decision trees (a.k.a. extra-trees) on various sub-samples of the dataset and uses 
# averaging to improve the predictive accuracy and control over-fitting

from sklearn.ensemble import ExtraTreesRegressor
import numpy as np

def initialize(context):
    
    # Select security
    context.security = sid(5938) # PG
    
    # Set commission and slippage
    set_commission(commission.PerShare(cost=0.05, min_trade_cost=1.00))
    set_slippage(slippage.VolumeShareSlippage(volume_limit=0.0025, price_impact=0.01))

    # Define model
    context.model = ExtraTreesRegressor(bootstrap = False,
                                        n_estimators = 20, 
                                        max_features = 'auto')
    
    context.lookback = 3 # 3-day lookback
    context.history_range = 252 * 2 # Past 2 years' history (avg. 252 trading days/yr.)
    
    # Generate new model every week (1 min. before market close)
    schedule_function(create_model, date_rules.week_end(), time_rules.market_close(minutes=1))

    # Trade every day, 30 mins. after market open
    schedule_function(trade, date_rules.every_day(), time_rules.market_open(minutes=30))
    
def handle_data(context, data):

    # Create trendlines  
    fields = ['price', 'volume']
    for field in fields:
      record(field, data.current(context.security,field))

def create_model(context, data):
    
    # Get daily prices and volumes
    recent_prices = data.history(context.security, 'price', context.history_range, '1d').values
    recent_volumes = data.history(context.security, 'volume', context.history_range, '1d').values
    
    # Get price and volume differences
    price_changes = np.diff(recent_prices).tolist()
    volume_changes = np.diff(recent_volumes).tolist()

    # Initialize variables
    X = []
    Y = []
    
    # Store price and volume changes
    for i in range(context.history_range-context.lookback-1):
        X.append(price_changes[i:i+context.lookback] + volume_changes[i:i+context.lookback])
        Y.append(price_changes[i+context.lookback] + volume_changes[i+context.lookback])

    # Generate model
    context.model.fit(X, Y)

def trade(context, data):
    
    # If model is generated:
    if context.model:
        
        # Get price and volume history
        recent_prices = data.history(context.security, 'price', context.lookback+1, '1d').values
        recent_volumes = data.history(context.security, 'volume', context.lookback+1, '1d').values
        
        # Calculate change in price and volume
        price_changes = np.diff(recent_prices).tolist()
        volume_changes = np.diff(recent_volumes).tolist()
        
        # Use price and volume changes to generate prediction, display
        prediction = context.model.predict(price_changes + volume_changes)
        record(prediction = prediction)
        
        # If price predicted to rise, long; if price predicted to fall, short (order 5%)
        if prediction > 0:
            order_target_percent(context.security, 5.0)
        else:
            order_target_percent(context.security, -5.0)